In [1]:
import numpy as np

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import h5py
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.applications.vgg16 import VGG16
# from keras.layers.normalization import BatchNormalization
# from keras.optimizers import Adam
from keras import optimizers
from keras.utils import np_utils
from keras.models import model_from_json
# from sklearn.metrics import log_loss
from numpy.random import permutation
from sklearn import preprocessing
from sklearn import utils

np.random.seed(2016)
use_cache = 1

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and fun

In [2]:
#以RGB形式获取一张图像并预处理（减去RGB均值）
def get_im(path, img_rows=224, img_cols=224):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    resized = cv2.resize(img, (img_cols, img_rows))
    mean_pixel = [113.68, 116.799, 103.939]
    
    for c in range(3):
        resized[:, :, c] = resized[:, :, c] - mean_pixel[c]

    return resized

In [3]:
#从driver_imgs_list.csv获取（图像文件名-司机id）字典
def get_driver_data():
    dr = dict()
    path = os.path.join('.', 'driver_imgs_list.csv')
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        dr[arr[2]] = arr[0]
    f.close()
    return dr

In [4]:
#以ndarray形式加载训练集,并对标签进行独热编码
def load_train(img_rows=224, img_cols=224):
    X_train = []
    y_train = []
    driver_id = []

    driver_data = get_driver_data()

    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join('.', 'train', 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im(fl, img_rows, img_cols)
            X_train.append(img)
            y_train.append(j)
            driver_id.append(driver_data[flbase])
    
    one_hot_encoder = preprocessing.OneHotEncoder(n_values=10)
    y_train = one_hot_encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
    unique_drivers = sorted(list(set(driver_id)))
    print('Unique drivers: {}'.format(len(unique_drivers)))
    print(unique_drivers)
    return np.array(X_train, dtype='float32'), np.array(y_train, dtype='float32'), driver_id, unique_drivers

In [5]:
#分离出某些司机数据作为验证集
def train_val_split(driver_val, driver_id, X, y):
    val = []
    tra = [a for a in range(len(X))]
    assert (len(driver_id) == len(X)) & (len(driver_id) == len(y))
    
    for a in range(len(X)):
        if driver_id[a] in driver_val:
            val.append(a)
    X_val = np.array([X[a] for a in val], dtype='float32')
    y_val = np.array([y[a] for a in val], dtype='float32')
    
    for a in val:
        tra.remove(a)
    X_train = np.array([X[a] for a in tra], dtype='float32')
    y_train = np.array([y[a] for a in tra], dtype='float32')
    
    return X_train, y_train, X_val, y_val

In [6]:
#以ndarray形式加载测试集
def load_test(img_rows=224, img_cols=224):
    print('Read test images')
    path = os.path.join('.', 'test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im(fl, img_rows, img_cols)
        X_test.append(img)
        X_test_id.append(flbase)
        total += 1
        if total % thr == 0:
            print('Read {} images from {}'.format(total, len(files)))

    return X_test, X_test_id

In [7]:
def cache_data(data, path):
    if not os.path.isdir('cache'):
        os.mkdir('cache')
    if os.path.isdir(os.path.dirname(path)):
        file = open(path, 'wb')
        pickle.dump(data, file)
        file.close()
    else:
        print('Directory doesnt exists')

In [8]:
def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        print('Restore data from pickle........')
        file = open(path, 'rb')
        data = pickle.load(file)
    return data

In [9]:
def read_train_and_val_data(img_rows, img_cols):

    cache_path = os.path.join('cache', 'train_r_' + str(img_rows) +
                              '_c_' + str(img_cols) + '_t_' +
                              str(3) + '.dat')

    if not os.path.isfile(cache_path) or use_cache == 0:
        X_train, y_train, driver_id, unique_drivers = load_train(img_rows, img_cols)
        cache_data((X_train, y_train, driver_id, unique_drivers), cache_path)
    else:
        print('Restore train from cache!')
        (X_train, y_train, driver_id, unique_drivers) = restore_data(cache_path)
    driver_val = ['p026','p035','p039','p041']
    X_train, y_train, X_val, y_val = train_val_split(driver_val, driver_id, X_train, y_train)
    print('Train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    return X_train, y_train, X_val, y_val

In [10]:
#对数据随机排序并划分batch
def shuffle_and_batch(batch_size, X_train, y_train):
    new_X_train, new_y_train = utils.shuffle(X_train, y_train)
    for start in range(0, len(new_X_train), batch_size):
        end = min(start + batch_size, len(X_train))
        yield np.array(new_X_train[start:end], dtype='float32'), np.array(new_y_train[start: end], dtype='float32')

In [11]:
def vgg_model():
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [12]:
X_train, y_train, driver_id, unique_drivers = load_train(224, 224)

Read drivers data
Read train images
Load folder c0
Load folder c1
Load folder c2
Load folder c3
Load folder c4
Load folder c5
Load folder c6
Load folder c7
Load folder c8
Load folder c9
Unique drivers: 26
['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']


In [13]:
driver_val = ['p026','p035','p039','p041']
X_train, y_train, X_val, y_val = train_val_split(driver_val, driver_id, X_train, y_train)

In [14]:
X_train, y_train = utils.shuffle(X_train, y_train)

In [15]:
X_train.shape

(19124, 224, 224, 3)

In [16]:
sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
vgg = vgg_model()
for layer in vgg.layers[:4]:
    layer.trainable = True
for layer in vgg.layers[4:]:
    layer.trainable = True

for i, layer in enumerate(vgg.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 flatten_1
20 dense_1
21 dense_2
22 dense_3


In [17]:
vgg.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist = vgg.fit(x=X_train, y=y_train, batch_size=64, epochs=10, verbose=1, validation_data=(X_val, y_val), shuffle=True)

Train on 19124 samples, validate on 3300 samples
Epoch 1/10
 6976/19124 [=========>....................] - ETA: 6:53 - loss: 14.3713 - acc: 0.1075

In [ ]:
#batch_size = 64
#steps_per_epoch = int(math.ceil(len(X_train)/batch_size))

In [ ]:
#hist = vgg.fit_generator(shuffle_and_batch(batch_size, X_train, y_train), steps_per_epoch, epochs=10, verbose=1)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model
import pydot
import graphviz
plot_model(vgg, show_shapes=True, to_file='model_simple.png')
SVG(model_to_dot(vgg, show_shapes=True).create(prog='dot', format='svg'))
